1. VERİ ÖNİŞLEME - R PROGRAMLAMA
2. VERİ GÖRSELLEŞTİRME - R PROGRAMLAMA
3. MAKİNE ÖĞRENMESİ ALGORİTMA MODELLERİ - PYHTON
4. ELDE EDİLEN SONUÇLARIN GÖRSELLEŞTİRİLMESİ - R PROGRAMLAMA

# **3. MAKİNE ÖĞRENMESİ ALGORİTMALARI**

In [ ]:
''' KÜTÜPHANELER '''
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')


In [ ]:
''' VERİ YÜKLEME '''
# Örnek veri yükleme (gerçek veri gizlilik nedeniyle paylaşılmamıştır)
# df = pd.read_csv("sample_data.csv") # yorum satırına al
import joblib

# dataframei kaydetme
#joblib.dump(df, 'dataframe_1.pkl')  # yorum satırına al

# dataframei yükleme
dff = joblib.load('dataframe_1.pkl')
dff

ValueError: EOF: reading array data, expected 262144 bytes got 117136

In [ ]:
# Değişken isimleri
print(dff.columns)


In [ ]:
# Sütun isimlerini seçiyoruz (Veri setini bize atıldığı haline getiriyoruz.)
selected_col = ['CARIKART_ID', 'FATURA_ID', 'FATURA_TARIHI',
       'BIRTHDATE_E', 'MALZEME_ID', 'MIKTAR', 'DTUTAR',
       'MALZEME_ADI', 'WEB_ANA_KATEGORI_TANIMI', 'WEB_BIRINCI_KATEGORI_TANIMI',
       'WEB_IKINCI_KATEGORI_TANIMI', 'WEB_UCUNCU_KATEGORI_TANIMI', 'BEDEN',
       'MARKA_ADI', 'RENK_ADI', 'TEKSTIL_CINSIYET']

dff = dff[selected_col]
dff

In [ ]:
''' Kayıp Gözlem Değerlerin İncelenmesi '''
import missingno as mso
import numpy as np

mso.bar(dff, color='lightblue')

In [ ]:
df.isnull().sum()

In [ ]:
#Kayıp gözlem değerlerini  doldur.
df['RENK_ADI'].fillna("Renksiz", inplace=True)
df['TEKSTIL_CINSIYET'].fillna("Cinsiyetsiz", inplace=True)
df['BEDEN'].fillna("(-3)-48", inplace=True)
df['WEB_IKINCI_KATEGORI_TANIMI'].fillna("Kategorisiz", inplace=True)

In [ ]:
''' Model Performansı artırmak için değişken seçimi '''
df= df[['CARIKART_ID', 'FATURA_ID',"MALZEME_ID","MALZEME_ADI",'WEB_BIRINCI_KATEGORI_TANIMI',"WEB_IKINCI_KATEGORI_TANIMI","BEDEN","MARKA_ADI","TEKSTIL_CINSIYET", "gun"]]
df

In [ ]:
# Kategorik değişkenler için frekans kodlaması yapılıyor.
for column in df.select_dtypes(include=['object']).columns:
    frequency_encoding = df[column].value_counts(normalize=True)
    df[column + '_Frequency_Encoded'] = df[column].map(frequency_encoding)

# Orijinal kategorik sütunlar alınmıyor
df = df.drop(df.select_dtypes(include=['object']).columns, axis=1)

print(df)

In [ ]:
# X ve y tanımlanıyor.
y = df["gun"]
X = df[['CARIKART_ID', 'FATURA_ID','MALZEME_ID_Frequency_Encoded',
       'MALZEME_ADI_Frequency_Encoded',
       'WEB_BIRINCI_KATEGORI_TANIMI_Frequency_Encoded',
       'WEB_IKINCI_KATEGORI_TANIMI_Frequency_Encoded',
       'BEDEN_Frequency_Encoded', 'MARKA_ADI_Frequency_Encoded',
       'TEKSTIL_CINSIYET_Frequency_Encoded']]

# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
X = pd.DataFrame(X_standardized, columns=X.columns)

In [ ]:
# train-test ayrımı
from sklearn.model_selection import train_test_split
X_train_D, X_test_D, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
# Sepete göre analiz yapmak istediğimiz için R kodu ve Pyhton kodlarını birleştirme sürecinde 'CARIKART_ID', 'FATURA_ID' değişkenlerini kullanabilmek için  X_train_D ve X_test_D olarak atama yapılmıştır.
X_train_D
print(X_train_D.columns)


In [ ]:
# Model performanslarını en iyi sonuç elde etmek için  X_train ve X_test e atanan değişkenler tekrar düzenleniyor.
#X_train
selected_columns = ['MALZEME_ID_Frequency_Encoded',
                    'MALZEME_ADI_Frequency_Encoded',
                    'WEB_BIRINCI_KATEGORI_TANIMI_Frequency_Encoded',
                    'WEB_IKINCI_KATEGORI_TANIMI_Frequency_Encoded',
                    'BEDEN_Frequency_Encoded',
                    'MARKA_ADI_Frequency_Encoded',
                    'TEKSTIL_CINSIYET_Frequency_Encoded']


X_train = X_train_D[selected_columns]

#X_test
selected_columns = ['MALZEME_ID_Frequency_Encoded',
                    'MALZEME_ADI_Frequency_Encoded',
                    'WEB_BIRINCI_KATEGORI_TANIMI_Frequency_Encoded',
                    'WEB_IKINCI_KATEGORI_TANIMI_Frequency_Encoded',
                    'BEDEN_Frequency_Encoded',
                    'MARKA_ADI_Frequency_Encoded',
                    'TEKSTIL_CINSIYET_Frequency_Encoded']


X_test = X_test_D[selected_columns]


In [ ]:
''' LİNEER REGRESYON'''
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
import numpy as np
# Lineer regresyon modelini oluştur
model = LinearRegression()

# Modeli eğit
model.fit(X_train, y_train)

# Eğitim setindeki tahminleri al
y_train_pred_lr = model.predict(X_train)

# Test setindeki tahminleri al
y_test_pred_lr = model.predict(X_test)

# MSE ve RMSE değerlerini hesapla
mse_train = mean_squared_error(y_train, y_train_pred_lr)
rmse_train = np.sqrt(mse_train)
mse_test = mean_squared_error(y_test, y_test_pred_lr)
rmse_test = np.sqrt(mse_test)

# Model performansını yazdır
print("LR Eğitim Seti R^2 Skoru:", r2_score(y_train, y_train_pred_lr))
print("LR Test Seti R^2 Skoru:", r2_score(y_test, y_test_pred_lr))
print("LR Eğitim Seti MSE:", mse_train)
print("LR Eğitim Seti RMSE:", rmse_train)
print("LR Test Seti MSE:", mse_test)
print("LR Test Seti RMSE:", rmse_test)

# Tahminler ve gerçek değerleri birleştirerek bir DataFrame oluştur
predictions_df = pd.DataFrame({'Gerçek Değerler': y_test, 'LR Tahminler': y_test_pred_lr})

# İlk 10 gözlemi göster
print(predictions_df.head(10))



In [ ]:
''' LİNEER REGRESYON DOĞRULUK ORANI '''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 60  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_lr)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani * 100:.2f}%")

In [ ]:
''' POLİNOMİYAL REGRESYON 4.DERECEDEN '''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


# Polinom özellikleri ekleyerek X'i genişlet
degree = 4  # İsteğe bağlı: polinom derecesi
poly = PolynomialFeatures(degree=degree)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Lineer regresyon modelini oluştur
model = LinearRegression()

# Modeli eğit
model.fit(X_train_poly, y_train)

# Eğitim setindeki tahminleri al
y_train_pred_pol4 = model.predict(X_train_poly)

# Test setindeki tahminleri al
y_test_pred_pol4 = model.predict(X_test_poly)

# MSE ve RMSE değerlerini hesapla
mse_train = mean_squared_error(y_train, y_train_pred_pol4)
rmse_train = np.sqrt(mse_train)
mse_test = mean_squared_error(y_test, y_test_pred_pol4)
rmse_test = np.sqrt(mse_test)

# Model performansını yazdır
print("POL4 Eğitim Seti RMSE:", rmse_train)
print("POL4 Eğitim Seti MSE:", mse_train)
print("POL4 Eğitim Seti R^2 Skoru:", r2_score(y_train, y_train_pred_pol4))

print("POL4 Test Seti RMSE:", rmse_test)
print("POL4 Test Seti MSE:", mse_test)
print("POL4 Test Seti R^2 Skoru:", r2_score(y_test, y_test_pred_pol4))

# Tahminler ve gerçek değerleri birleştirerek bir DataFrame oluştur
predictions_df = pd.DataFrame({'Gerçek Değerler': y_test, 'POL4 Tahminler': y_test_pred_pol4})

# İlk 10 gözlemi göster
print(predictions_df.head(10))

In [ ]:
''' POLİNOMİYAL REGRESYON 4.DERECEDEN DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 60  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_pol4)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
''' RANDOM FOREST REGRESYON '''
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np


# Random Forest regresyon modelini oluşturma ve eğitme
model = RandomForestRegressor(n_estimators= 192)

model.fit(X_train, y_train)#yorum satırına al

import joblib

# Eğitilmiş modeli kaydetme
joblib.dump(model, 'egitilmis_model.pkl_rf')  # yorum satırına al

# Kaydedilen modeli yükleme
model = joblib.load('egitilmis_model.pkl_rf')

# Test verilerini kullanarak tahmin yapma
y_test_pred_rf = model.predict(X_test)

# RMSE hesapla
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))

# TEST setindeki R² Skoru hesapla
r2_score = model.score(X_test, y_test)

mae = mean_absolute_error(y_test, y_test_pred_rf)
# Tahminler ve gerçek değerleri birleştirerek bir DataFrame oluştur
predictions_df = pd.DataFrame({'Gerçek Değerler': y_test, 'RF Tahminler': y_test_pred_rf})

# Eğitim seti üzerinde tahmin yapma
y_train_pred_rf = model.predict(X_train)

# Eğitim setindeki RMSE hesapla
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))

# Eğitim setindeki R² Skoru hesapla
r2_train = model.score(X_train, y_train)

# Eğitim setindeki Mean Absolute Error hesapla
mae_train = mean_absolute_error(y_train, y_train_pred_rf)

# Sonuçları yazdır
print("RF Test Seti RMSE:", rmse)
print("RF Test Seti R² Score:", r2_score)
print("RF Test Seti MAE:", mae)


print("RF Eğitim Seti  RMSE:", rmse_train)
print("RF Eğitim Seti R² Score:", r2_train)
print("RF Eğitim Seti MAE:", mae_train)

# İlk 10 gözlemi göster
print(predictions_df.head(10))


In [ ]:
''' RANDOM FOREST REGRESYON  DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 90  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_rf)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
#Tahmin değerleri tam sayı yapılıyor
y_test_pred_rf = np.round(y_test_pred_rf)
y_test_pred_rf

In [ ]:
''' MÜŞTERİELERİN FATURA_ID GÖRE GRUPLANDIRMA '''
import pandas as pd
from sklearn.metrics import median_absolute_error


# Tahminleri içeren bir DataFrame oluşturun
sepet_sonuclar = pd.DataFrame({
    'FATURA_ID': X_test_D['FATURA_ID'],  # X_test içinde yer alan FATURA_ID değişkeni sepet_sonuclar içine aktarılıyor
    'CARIKART_ID': X_test_D['CARIKART_ID'],  # X_test içinde yer alan CARIKART_ID değişkeni sepet_sonuclar içine aktarılıyor
    'günlük': y_test,  # yanıt değişkeni atanıyor
    'tahmin': y_test_pred_rf #RF tahmin sonucları ekleniyor
})

# Sepeti özetleme işlemlerini gerçekleştirin
sepet_sonuclar = sepet_sonuclar.groupby('FATURA_ID').agg(
    CARIKART_ID=('CARIKART_ID', 'first'),
    günlük=('günlük', 'first'),
    medyan_tahmin=('tahmin', 'median'),
    ortalama_tahmin=('tahmin', 'mean'),
    adet=('tahmin', 'size')
).reset_index()

# İki sütun arasındaki eşleşmeyi kontrol etme
eşleşme_oranı_medyan = (sepet_sonuclar['medyan_tahmin'] == sepet_sonuclar['günlük']).mean()
eşleşme_oranı_ortalama = (sepet_sonuclar['ortalama_tahmin'] == sepet_sonuclar['günlük']).mean()

# Sonuçları görüntüleme
print("Medyan Tahmin Eşleşme Oranı:", eşleşme_oranı_medyan)
print("Ortalama Tahmin Eşleşme Oranı:", eşleşme_oranı_ortalama)

# Özetlenmiş sepet sonuçlarını gösterme
print("Sepet Sonuçları:")
print(sepet_sonuclar)


In [ ]:
''' DECİSİON TREE '''
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

# Karar ağacı regresyon modelini oluştur
reg_model = DecisionTreeRegressor(splitter='best', min_samples_split=2, min_samples_leaf=4, max_depth=30, criterion='friedman_mse')

# Modeli eğitim verisi üzerinde eğit
reg_model.fit(X_train, y_train)

# Eğitim seti üzerinde tahmin yap
y_train_pred_dt = reg_model.predict(X_train)

# Eğitim seti üzerinde MSE değerini hesapla
train_mse = mean_squared_error(y_train, y_train_pred_dt)

# Test seti üzerinde tahmin yap
y_test_pred_dt = reg_model.predict(X_test)

# Test seti üzerinde MSE değerini hesapla
test_mse = mean_squared_error(y_test, y_test_pred_dt)

# Tahminler ve gerçek değerleri birleştirerek bir DataFrame oluştur
predictions_df = pd.DataFrame({'Gerçek Değerler': y_test, 'DT Tahminler': y_test_pred_dt})


# Eğitim seti üzerinde RMSE, MAE ve R-squared değerlerini hesapla
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred_dt)
train_r2 = r2_score(y_train, y_train_pred_dt)
print("DT Eğitim Seti MSE:", train_mse)
print("DT Eğitim Seti - RMSE:", train_rmse)
print("DT Eğitim Seti - MAE:", train_mae)
print("DT Eğitim Seti - R-squared (R^2):", train_r2)

# Test seti üzerinde RMSE, MAE ve R-squared değerlerini hesapla
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred_dt)
test_r2 = r2_score(y_test, y_test_pred_dt)

print("DT Test Seti MSE:", test_mse)
print("DT Test Seti - RMSE:", test_rmse)
print("DT Test Seti - MAE:", test_mae)
print("DT Test Seti - R-squared (R^2):", test_r2)

# İlk 10 gözlemi göster
print(predictions_df.head(10))

In [ ]:
''' DECİSİON TREE REGRESYON  DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 60  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_dt)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
''' KNN REGRESYON '''
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# KNN Regressor modelini oluştur
knn_regressor = KNeighborsRegressor(weights='distance' , p =1, n_neighbors=11)

# Modeli eğitim seti üzerinde eğit
knn_regressor.fit(X_train, y_train)

# Eğitim seti üzerinde tahmin yap
y_train_pred_knn = knn_regressor.predict(X_train)

# Eğitim seti üzerinde RMSE, MAE ve R-squared değerlerini hesapla
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_knn))
train_mae = mean_absolute_error(y_train, y_train_pred_knn)
train_r2 = r2_score(y_train, y_train_pred_knn)

print("KNN Eğitim Seti - RMSE:", train_rmse)
print("KNN Eğitim Seti - MAE:", train_mae)
print("KNN Eğitim Seti - R-squared (R^2):", train_r2)

# Test seti üzerinde tahmin yap
y_test_pred_knn = knn_regressor.predict(X_test)

# Test seti üzerinde RMSE, MAE ve R-squared değerlerini hesapla
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_knn))
test_mae = mean_absolute_error(y_test, y_test_pred_knn)
test_r2 = r2_score(y_test, y_test_pred_knn)

print("KNN Test Seti - RMSE:", test_rmse)
print("KNN Test Seti - MAE:", test_mae)
print("KNN Test Seti - R-squared (R^2):", test_r2)


#Tahminler ve gerçek değerleri birleştirerek bir DataFrame oluştur
predictions_df = pd.DataFrame({'Gerçek Değerler': y_test, 'KNN Tahminler': y_test_pred_knn})

# İlk 10 gözlemi göster
print(predictions_df.head(10))

In [ ]:
''' KNN REGRESYON  DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 60  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_knn)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
''' XGB REGRESYON '''

# Gerekli kütüphaneyi yükleyin
# Eğer yüklü değilse: pip install xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# XGBoost Regressor modelini oluşturun ve eğitin
model = xgb.XGBRegressor(objective ='reg:squarederror', seed=42)
model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapın
y_test_pred_xgb = model.predict(X_test)

# Test seti üzerinde RMSE, MAE ve R-squared değerlerini hesapla
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
test_mae = mean_absolute_error(y_test, y_test_pred_xgb)
test_r2 = r2_score(y_test, y_test_pred_xgb)


# Train seti üzerinde tahmin yapın
y_train_pred_xgb = model.predict(X_train)


# Eğitim seti üzerinde RMSE, MAE ve R-squared değerlerini hesapla
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_xgb))
train_mae = mean_absolute_error(y_train, y_train_pred_xgb)
train_r2 = r2_score(y_train, y_train_pred_xgb)

print("XGB Eğitim Seti - RMSE:", train_rmse)
print("XGB Eğitim Seti - MAE:", train_mae)
print("XGB Eğitim Seti - R-squared (R^2):", train_r2)




print("XGB Test Seti - RMSE:", test_rmse)
print("XGB Test Seti - MAE:", test_mae)
print("XGB Test Seti - R-squared (R^2):", test_r2)


#Tahminler ve gerçek değerleri birleştirerek bir DataFrame oluştur
predictions_df = pd.DataFrame({'Gerçek Değerler': y_test, 'XGB Tahminler': y_test_pred_xgb})

# İlk 10 gözlemi göster
print(predictions_df.head(10))


In [ ]:
''' XGB REGRESYON  DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 90  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_xgb)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
''' MÜŞTERİELERİN FATURA_ID GÖRE GRUPLANDIRMA '''

import pandas as pd
from sklearn.metrics import median_absolute_error

# Tahminleri içeren bir DataFrame oluşturun
sepet_sonuclar = pd.DataFrame({
    'FATURA_ID': X_test_D['FATURA_ID'],  # X_test içinde yer alan FATURA_ID değişkeni sepet_sonuclar içine aktarılıyor
    'CARIKART_ID': X_test_D['CARIKART_ID'],  # X_test içinde yer alan CARIKART_ID değişkeni sepet_sonuclar içine aktarılıyor
    'günlük': y_test,  # yanıt değişkeni atanıyor
    'tahmin': y_test_pred_xgb #xgb tahmin sonucları ekleniyor
})

# Sepeti özetleme işlemlerini gerçekleştirin
sepet_sonuclar = sepet_sonuclar.groupby('FATURA_ID').agg(
    CARIKART_ID=('CARIKART_ID', 'first'),
    günlük=('günlük', 'first'),
    medyan_tahmin=('tahmin', 'median'),
    ortalama_tahmin=('tahmin', 'mean'),
    adet=('tahmin', 'size')
).reset_index()

# İki sütun arasındaki eşleşmeyi kontrol etme
eşleşme_oranı_medyan = (sepet_sonuclar['medyan_tahmin'] == sepet_sonuclar['günlük']).mean()
eşleşme_oranı_ortalama = (sepet_sonuclar['ortalama_tahmin'] == sepet_sonuclar['günlük']).mean()

# Sonuçları görüntüleme
print("Medyan Tahmin Eşleşme Oranı:", eşleşme_oranı_medyan)
print("Ortalama Tahmin Eşleşme Oranı:", eşleşme_oranı_ortalama)

# Özetlenmiş sepet sonuçlarını gösterme
print("Sepet Sonuçları:")
print(sepet_sonuclar)


In [ ]:
sepet_sonuclar.to_excel("sepet_sonucc.xlsx")

In [ ]:
sepet_sonucc=pd.read_excel("sepet_sonucc_xgb.xlsx")

In [ ]:
sepet_sonucc = np.round(sepet_sonucc)
sepet_sonucc

In [ ]:
sepet_sonucc.to_excel("sepet_sonucc__NET_XGB.xlsx")

# **Yeni Data Yükleniyor**

In [ ]:
df_test= pd.read_excel('estu_test_data3.xlsx')
df_test

In [ ]:
X_test_N= df_test[["MALZEME_ID","MALZEME_ADI",'WEB_BIRINCI_KATEGORI_TANIMI',"WEB_IKINCI_KATEGORI_TANIMI","BEDEN","MARKA_ADI","TEKSTIL_CINSIYET"]]
X_test_N


In [ ]:
print(X_test_N.dtypes)


In [ ]:
#Kayıp gözlemler dolduruluyor.
X_test_N['TEKSTIL_CINSIYET'].fillna("Cinsiyetsiz", inplace=True)
X_test_N['WEB_IKINCI_KATEGORI_TANIMI'].fillna("Kategorisiz", inplace=True)
X_test_N['BEDEN'].fillna("(-3)-48", inplace=True)
X_test_N['MALZEME_ADI'].fillna("bilinmiyor", inplace=True)
X_test_N['MARKA_ADI'].fillna("bilinmiyor", inplace=True)


In [ ]:
X_test_N.isnull().sum()

In [ ]:
# Kategorik değişkenler için frekans kodlaması yapılıyor.
for column in X_test_N.select_dtypes(include=['object']).columns:
    frequency_encoding = X_test_N[column].value_counts(normalize=True)
    X_test_N[column + '_Frequency_Encoded'] = X_test_N[column].map(frequency_encoding)

# Orijinal kategorik sütunları bırak
X_test_N = X_test_N.drop(X_test_N.select_dtypes(include=['object']).columns, axis=1)

print(X_test_N)

In [ ]:
print(X_test_N.dtypes)


In [ ]:
# Standarization

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Önce sütun isimlerini bir değişkende saklayın
column_names = X_test_N.columns

# Diziyi Standart Ölçeklendirmeye tabi tutun
scaler = StandardScaler()
X_test_N = scaler.fit_transform(X_test_N)

# Standart Ölçeklenmiş veriyi DataFrame'e çevirin ve sütun isimlerini atayın
X_test_N = pd.DataFrame(X_test_N, columns=column_names)


**YENİ VERİ İLE KURULAN MODELLERİN TAHMİNLERİ**


In [ ]:
X_train

In [ ]:
''' LİNEER REGRESYON'''
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
import numpy as np
# Lineer regresyon modelini oluştur
model = LinearRegression()

# Modeli eğit
model.fit(X_train, y_train)

# Test setindeki tahminleri al
y_test_pred_lr_N = model.predict(X_test_N)


In [ ]:
''' LİNEER REGRESYON DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 90  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_lr_N)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

Doğruluk Oranı: 22.19%


In [ ]:
#Tahmin değerlerei tam sayı olarak cevriliyor.
y_test_pred_lr_N = np.round(y_test_pred_lr_N)
df_test['LineerR_tahmin'] = y_test_pred_lr_N


In [ ]:
y_test_pred_lr_N

In [ ]:
''' POLİNOMİYAL REGRESYON 4.DERECEDEN '''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


# Polinom özellikleri ekleyerek X'i genişlet
degree = 4  # İsteğe bağlı: polinom derecesi
poly = PolynomialFeatures(degree=degree)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test_N)

# Lineer regresyon modelini oluştur
model = LinearRegression()

# Modeli eğit
model.fit(X_train_poly, y_train)

# Test setindeki tahminleri al
y_test_pred_pol4_N = model.predict(X_test_poly)


In [ ]:
''' POL4 REGRESYON DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 90  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_pol4_N)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
#Tahmin değerleri tam sayı olarak yazılıyor.
y_test_pred_pol4_N = np.round(y_test_pred_pol4_N)
df_test['POL4_tahmin'] = y_test_pred_pol4_N


In [ ]:
''' RANDOM FOREST REGRESYON '''
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error

import numpy as np


# Random Forest regresyon modelini oluşturma ve eğitme
model = RandomForestRegressor(n_estimators= 192)


#model.fit(X_train, y_train)#yorum satırına al

import joblib

# Eğitilmiş modeli kaydetme
#joblib.dump(model, 'egitilmis_model.pkl_rf')  # yorum satırına al

# Kaydedilen modeli yükleme
model = joblib.load('egitilmis_model.pkl_rf')

# Test verilerini kullanarak tahmin yapma
y_test_pred_rf_N = model.predict(X_test_N)



In [ ]:
''' RF REGRESYON DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 60  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_rf_N)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
#Tahmin değerleri tam sayı olarak yazılıyor.
y_test_pred_rf_N = np.round(y_test_pred_rf_N)
df_test['RandomF_tahmin'] = y_test_pred_rf_N


In [ ]:
y_test_pred_rf_N

In [ ]:
''' DECİSİON TREE '''
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

# Karar ağacı regresyon modelini oluştur
reg_model = DecisionTreeRegressor(splitter='best', min_samples_split=2, min_samples_leaf=4, max_depth=30, criterion='friedman_mse')

# Modeli eğitim verisi üzerinde eğit
reg_model.fit(X_train, y_train)


# Test seti üzerinde tahmin yap
y_test_pred_dt_N = reg_model.predict(X_test_N)



In [ ]:
''' DECİSİON TREE REGRESYON  DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 60  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_dt_N)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
#Tahmin değerleri tam sayı olarak yazılıyor.
y_test_pred_dt_N = np.round(y_test_pred_dt_N)
df_test['DecisionT_tahmin'] = y_test_pred_dt_N


In [ ]:
''' KNN REGRESYON '''
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# KNN Regressor modelini oluştur
knn_regressor = KNeighborsRegressor(weights='distance' , p =1, n_neighbors=11)

# Modeli eğitim seti üzerinde eğit
knn_regressor.fit(X_train, y_train)


# Test seti üzerinde tahmin yap
y_test_pred_knn_N = knn_regressor.predict(X_test_N)


In [ ]:
''' KNN REGRESYON  DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 90  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_knn_N)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
#Tahmin değerleri tam sayı olarak yazılıyor.
y_test_pred_knn_N = np.round(y_test_pred_knn_N)
df_test['KNN_tahmin'] = y_test_pred_knn_N


In [ ]:
''' XGB REGRESYON '''

# Gerekli kütüphaneyi yükleyin
# Eğer yüklü değilse: pip install xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# XGBoost Regressor modelini oluşturun ve eğitin
model = xgb.XGBRegressor(objective ='reg:squarederror', seed=42)
model.fit(X_train, y_train)

# Test seti üzerinde tahmin yapın
y_test_pred_xgb_N = model.predict(X_test_N)



In [ ]:
''' XGB REGRESYON  DOĞRULUK ORANI'''

def dogruluk_olcumu(gercek_deger, tahmin_edilen_deger, marj):
    alt_sinir = gercek_deger - marj
    ust_sinir = gercek_deger + marj

    if tahmin_edilen_deger >= alt_sinir and tahmin_edilen_deger <= ust_sinir:
        return True
    else:
        return False

marj = 90  # Kabul edilebilir marj

# Her bir eleman için doğruluk ölçümünü yapma
dogruluk_sonuclari = [dogruluk_olcumu(gercek, tahmin, marj) for gercek, tahmin in zip(y_test, y_test_pred_xgb_N)]

# Doğru tahminlerin sayısını ve toplam veri sayısını hesaplama
dogru_tahmin_sayisi = sum(dogruluk_sonuclari)
toplam_veri_sayisi = len(dogruluk_sonuclari)
dogruluk_orani = dogru_tahmin_sayisi / toplam_veri_sayisi

print(f"Doğruluk Oranı: {dogruluk_orani*100:.2f}%")

In [ ]:
#Tahmin değerleri tam sayı olarak yazılıyor.
y_test_pred_xgb_N = np.round(y_test_pred_xgb_N)
df_test['XGB_tahmin'] = y_test_pred_xgb_N


In [ ]:
df_test.to_excel("df_test_tahminler.xlsx")